# Pràctica 3

In [1]:
import nltk
nltk.download('conll2002')
from nltk.corpus import conll2002
train_es = conll2002.iob_sents('esp.train') # Train
dev_es = conll2002.iob_sents('esp.testa') # Dev
test_es =conll2002.iob_sents('esp.testb') # Test

train_ned = conll2002.iob_sents('ned.train') # Train
dev_ned = conll2002.iob_sents('ned.testa') # Dev
test_ned =conll2002.iob_sents('ned.testb') # Test

data = {'spanish': (train_es, dev_es, test_es),
        'dutch': (train_ned, dev_ned, test_ned)}

[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\11ser\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


In [2]:
train = conll2002.tagged_sents('esp.train')
test = conll2002.tagged_sents('esp.testb')
print(len(train))

8323


In [3]:
# Initialize the CRFTagger
model = nltk.tag.CRFTagger()


In [4]:
train_sample = train[0:500] #temps d'entrenament còmicament gran amb totes les dades: pillo un sample més petit
print(len(train_sample))

500


In [5]:
model.train(train, 'crfTaggerEs.mdl')


KeyboardInterrupt: 

In [ ]:
model.accuracy(test)

0.9549997089243786

In [6]:
def generar_features(tokens, idx, features_selection):
    """
    Genera una llista de característiques per a un token específic basat en les característiques seleccionades.

    features_selection és un diccionari que indica quines característiques incloure (True o False).

    Retorna una llista de característiques per al token donat.
    """
    caracteristiques = []
    caracteristiques.append('bias')

    token_actual = tokens[idx]

    # ------ FORMA DE LA PARAULA ------
    if features_selection.get('word_form', True):
        caracteristiques.extend([
            f'paraula={token_actual}',
            f'token_minuscula={token_actual.lower()}'
        ])

    # ------ PREFIXOS I SUFIXOS ------
    if features_selection.get('prefix_suffix', True):
        caracteristiques.extend([
            f'prefix-1={token_actual[0]}',
            f'prefix-2={token_actual[:2]}',
            f'prefix-3={token_actual[:3]}',
            f'sufix-1={token_actual[-1]}',
            f'sufix-2={token_actual[-2:]}',
            f'sufix-3={token_actual[-3:]}'
        ])

    # ------ MORFOLOGIA ------
    if features_selection.get('morphology', True):
        caracteristiques.extend([
            f'està_capitalitzat={token_actual[0].isupper()}',
            f'es_majuscules={token_actual.isupper()}',
            f'es_minuscules={token_actual.islower()}',
            f'té_guion={"-" in token_actual}',
            f'es_numeric={token_actual.isdigit()}',
            f'capitals_internes={token_actual[1:].lower() != token_actual[1:]}'
        ])

    # ------ LONGITUD ------
    if features_selection.get('length', True):
        caracteristiques.append(f'longitud={len(token_actual)}')

    # ------ POSICIÓ ------
    if features_selection.get('position', True):
        caracteristiques.extend([
            f'es_primer={idx == 0}',
            f'es_ultim={idx == len(tokens) - 1}'
        ])

    # ------ CONTEXT ANTERIOR ------
    if idx > 0 and features_selection.get('context', True):
        token_anterior = tokens[idx - 1]
        caracteristiques.extend([
            f'token_anterior={token_anterior.lower()}',
            f'anterior_capitalitzat={token_anterior[0].isupper()}',
            f'anterior_majuscules={token_anterior.isupper()}',
            f'anterior_minuscules={token_anterior.islower()}'
        ])

    # ------ CONTEXT POSTERIOR ------
    if idx < len(tokens) - 1 and features_selection.get('context', True):
        token_seguent = tokens[idx + 1]
        caracteristiques.extend([
            f'token_seguent={token_seguent.lower()}',
            f'seguent_capitalitzat={token_seguent[0].isupper()}',
            f'seguent_majuscules={token_seguent.isupper()}',
            f'seguent_minuscules={token_seguent.islower()}'
        ])

    return caracteristiques


In [8]:
feature_types = {
        'word_form': True,
        'lemma_pos_tags': False,
        'prefix_suffix': True,
        'morphology': True,
        'length': True,
        'position': True,
        'context': True
    }
ct = nltk.tag.CRFTagger(feature_func=generar_features(feature_types))
ct.train(train[:100], "nooooooolapolitziiaa.mdl")

TypeError: generar_features() missing 2 required positional arguments: 'idx' and 'features_selection'

In [9]:
def generate_features(feature_selection):
    def feature_function(sentence, index):
        features = {}
        word = sentence[index][0]  # Always get the first element (word)
        pos = sentence[index][1] if len(sentence[index]) > 1 else None  # Get POS if available
        
        # Características básicas de la palabra
        if feature_selection.get("word_form", True):
            features.update({
                "word": word,
                "word.lower": word.lower(),
                "word.istitle": word.istitle(),
                "word.isupper": word.isupper(),
                "word.isdigit": word.isdigit(),
            })
        
        # POS tagging y lemas
        if feature_selection.get("lemma_pos_tags", True) and pos:
            features["pos"] = pos
        
        # Prefijos y sufijos
        if feature_selection.get("prefix_suffix", True):
            features.update({
                "prefix3": word[:3],
                "suffix3": word[-3:],
                "prefix2": word[:2],
                "suffix2": word[-2:],
            })
        
        # Características morfológicas
        if feature_selection.get("morphology", True):
            features.update({
                "hyphen": "-" in word,
                "has_digit": any(c.isdigit() for c in word),
                "shape": "".join([
                    "X" if c.isupper() else 
                    "x" if c.islower() else 
                    "d" if c.isdigit() else c 
                    for c in word
                ])
            })
        
        # Longitud de la palabra
        if feature_selection.get("length", True):
            features["length"] = len(word)
        
        # Posición en la oración
        if feature_selection.get("position", True):
            features.update({
                "position": index,
                "is_first": index == 0,
                "is_last": index == len(sentence)-1
            })
        
        # Contexto circundante
        if feature_selection.get("context", True):
            if index > 0:
                prev_word = sentence[index-1][0]
                features.update({
                    "prev_word": prev_word,
                    "prev_word.lower": prev_word.lower(),
                    "prev_word.istitle": prev_word.istitle(),
                })
            if index < len(sentence)-1:
                next_word = sentence[index+1][0]
                features.update({
                    "next_word": next_word,
                    "next_word.lower": next_word.lower(),
                    "next_word.istitle": next_word.istitle(),
                })
        
        # Características adicionales
        features.update({
            "bias": 1.0,  # Término de sesgo
            "word.isalnum": word.isalnum(),
            "capital_inside": word[1:].lower() != word[1:] if len(word) > 1 else False
        })
        
        return features
    
    return feature_function

In [10]:
feature_selection = {
    "word_form": True,           # Basic word characteristics
    "prefix_suffix": True,       # Prefixes and suffixes
    "morphology": True,          # Morphological features
    "context": True,             # Surrounding words
    "extended_context": True,    # Wider context window (±2)
    "language_specific": True,   # Spanish/Dutch specific features
    "sentence_position": True,   # Position in sentence features
    "case_patterns": True,       # Capitalization patterns
    "date_time_patterns": True,  # Date and time patterns
    "numeric_patterns": True,    # Numeric features
    "symbol_patterns": True,     # Special characters
    "length": True,              # Word length features
    "orthographic_features": True, # Complex orthographic patterns
    "position": True,            # Position features
    "suffix_long": True,         # Longer suffixes (4+ chars)
    "common_ending": True        # Common word endings
}

ct = nltk.tag.CRFTagger(feature_func=generate_features(feature_selection))
ct.train(train[:100], "nooooooolapolitziiaa.mdl")
ct.accuracy(test)

0.7088467583878292

In [ ]:
def bio_to_io(tagged_sents):
    """
    Convert BIO tagging to IO tagging.
    
    In IO format, we don't distinguish between beginning (B-) and inside (I-) of an entity.
    All entity tokens are labeled as I-X.
    
    Args:
        tagged_sents: List of sentences with BIO tags
        
    Returns:
        List of sentences with IO tags
    """
    io_tagged_sents = []
    
    for sent in tagged_sents:
        io_sent = []
        for token in sent:
            word = token[0]  # Get the word
            tag = token[-1]  # Get the BIO tag
            
            # Convert B- to I- (Beginning to Inside)
            if tag.startswith('B-'):
                tag = 'I-' + tag[2:]
                
            io_sent.append((word, tag))
        io_tagged_sents.append(io_sent)
    
    return io_tagged_sents

# Example usage
printable_example = train_es[0][:10]  # First 10 tokens of first sentence
print("Original BIO tags:")
for token in printable_example:
    print(f"{token[0]} -> {token[-1]}")

# Convert to IO
io_example = bio_to_io([printable_example])[0]
print("\nConverted to IO tags:")
for token in io_example:
    print(f"{token[0]} -> {token[1]}")

Original BIO tags:
Melbourne -> B-LOC
( -> O
Australia -> B-LOC
) -> O
, -> O
25 -> O
may -> O
( -> O
EFE -> B-ORG
) -> O

Converted to IO tags:
Melbourne -> I-LOC
( -> O
Australia -> I-LOC
) -> O
, -> O
25 -> O
may -> O
( -> O
EFE -> I-ORG
) -> O


In [ ]:
def bio_to_biow(tagged_sents):
    """
    Convert BIO tagging to BIOW tagging.
    
    In BIOW format, we distinguish between:
    - B-X: Beginning of multi-token entity
    - I-X: Inside of entity
    - W-X: Single-token entity (Word)
    - O: Outside any entity
    
    Args:
        tagged_sents: List of sentences with BIO tags
        
    Returns:
        List of sentences with BIOW tags
    """
    biow_tagged_sents = []
    
    for sent in tagged_sents:
        biow_sent = []
        i = 0
        while i < len(sent):
            word = sent[i][0]  # Get the word
            tag = sent[i][-1]  # Get the BIO tag
            
            # If it's a B- tag, check if it's a single token entity
            if tag.startswith('B-'):
                entity_type = tag[2:]  # Get entity type (PER, LOC, etc.)
                
                # Check if next token continues this entity
                if i + 1 < len(sent) and sent[i+1][-1] == f'I-{entity_type}':
                    # Multi-token entity, keep as B-
                    biow_sent.append((word, tag))
                else:
                    # Single token entity, convert to W-
                    biow_sent.append((word, f'W-{entity_type}'))
            else:
                # Keep I- and O tags as they are
                biow_sent.append((word, tag))
                
            i += 1
            
        biow_tagged_sents.append(biow_sent)
    
    return biow_tagged_sents

# Example usage
print("\nConverted to BIOW tags:")
biow_example = bio_to_biow([printable_example])[0]
for token in biow_example:
    print(f"{token[0]} -> {token[1]}")


Converted to BIOW tags:
Melbourne -> W-LOC
( -> O
Australia -> W-LOC
) -> O
, -> O
25 -> O
may -> O
( -> O
EFE -> W-ORG
) -> O


In [ ]:
def convert_tagging_scheme(tagged_sents, source_scheme='bio', target_scheme='io'):
    """
    Convert between different tagging schemes.
    
    Args:
        tagged_sents: List of sentences with tags
        source_scheme: Source tagging scheme ('bio', 'io', 'biow')
        target_scheme: Target tagging scheme ('bio', 'io', 'biow')
        
    Returns:
        List of sentences with converted tags
    """
    # First convert to BIO if needed (not implemented here)
    if source_scheme.lower() != 'bio':
        # You would need to implement IO to BIO and BIOW to BIO converters
        raise NotImplementedError(f"Conversion from {source_scheme} to BIO not implemented")
    
    # Then convert from BIO to target scheme
    if target_scheme.lower() == 'io':
        return bio_to_io(tagged_sents)
    elif target_scheme.lower() == 'biow':
        return bio_to_biow(tagged_sents)
    elif target_scheme.lower() == 'bio':
        return tagged_sents  # No conversion needed
    else:
        raise ValueError(f"Unknown target scheme: {target_scheme}")

# Convert a sample of the training data to both schemes
sample_sents = train_es[:5]  # First 5 sentences
io_sents = convert_tagging_scheme(sample_sents, 'bio', 'io')
biow_sents = convert_tagging_scheme(sample_sents, 'bio', 'biow')

print(f"Converted {len(sample_sents)} sentences to IO and BIOW schemes")

# Count tag occurrences in each scheme
def count_tags(tagged_sents):
    tag_counts = {}
    for sent in tagged_sents:
        for token in sent:
            tag = token[-1]
            tag_counts[tag] = tag_counts.get(tag, 0) + 1
    return tag_counts

bio_counts = count_tags(sample_sents)
io_counts = count_tags(io_sents)
biow_counts = count_tags(biow_sents)

print("\nBIO tag counts:")
for tag, count in sorted(bio_counts.items()):
    print(f"{tag}: {count}")

print("\nIO tag counts:")
for tag, count in sorted(io_counts.items()):
    print(f"{tag}: {count}")

print("\nBIOW tag counts:")
for tag, count in sorted(biow_counts.items()):
    print(f"{tag}: {count}")

Converted 5 sentences to IO and BIOW schemes

BIO tag counts:
B-LOC: 4
B-MISC: 2
B-ORG: 2
B-PER: 3
I-ORG: 1
I-PER: 5
O: 146

IO tag counts:
I-LOC: 4
I-MISC: 2
I-ORG: 3
I-PER: 8
O: 146

BIOW tag counts:
B-ORG: 1
B-PER: 3
I-ORG: 1
I-PER: 5
O: 146
W-LOC: 4
W-MISC: 2
W-ORG: 1


In [ ]:
# Example: Converting all training and test data

# You would typically convert all your data like this:
def prepare_converted_data(language='spanish', target_scheme='io'):
    train, dev, test = data[language]
    
    # Convert to target scheme
    train_converted = convert_tagging_scheme(train, 'bio', target_scheme)
    dev_converted = convert_tagging_scheme(dev, 'bio', target_scheme)
    test_converted = convert_tagging_scheme(test, 'bio', target_scheme)
    
    return train_converted, dev_converted, test_converted

# For demonstration, we'll just convert a small sample
train_io, dev_io, test_io = prepare_converted_data('spanish','io')
train_biow, dev_biow, test_biow = prepare_converted_data('spanish','biow')




model_io = nltk.tag.CRFTagger(feature_func=generate_features(feature_selection))
model_io.train(train_io, "crfTaggerEs_IO.mdl")
model_io.accuracy(test_io)
# 
model_biow = nltk.tag.CRFTagger(feature_func=generate_features(feature_selection))
model_biow.train(train_biow, "crfTaggerEs_BIOW.mdl")
model_biow.accuracy(test_biow)

0.01758096753536569

# Extra: CADEC Corpus

In [ ]:
from nltk.corpus.reader import ConllCorpusReader

# Define column mappings (adjust to your CoNLL format)
column_types = ['words', 'pos', 'ignore', 'ne']  # Example for POS/NER

# Load the file
train = ConllCorpusReader(
    './cadec',           # Directory path
    'train.conll',       # Filename
    columntypes=column_types
)
test = ConllCorpusReader(
    './cadec',           # Directory path
    'test.conll',        # Filename
    columntypes=column_types
)

# Access data - Using appropriate methods
# Getting tagged sentences (word, tag pairs)
train_tagged_sents = train.tagged_sents()
test_tagged_sents = test.tagged_sents()
"""
# Print sample information to verify
print(f"Number of train sentences: {len(train_tagged_sents)}")
print(f"Number of test sentences: {len(test_tagged_sents)}")
print("\nSample from training data:")
if train_tagged_sents:
    print(train_tagged_sents[0][:10])  # First 10 tokens of first sentence

# Get words from the corpus
words = train.words()
print(f"\nNumber of words in training data: {len(words)}")
print(f"Sample words: {words[:20]}")  # First 20 words

# If you need NER tags specifically, you can extract them from tagged_sents
# This assumes your NE tags are in the format you specified in column_types
"""

ValueError: Inconsistent number of columns:
LIPITOR.408
pain	B-10033446	O	O	O	O
in	I-10033446	O	O	O	O
my	I-10033446	O	O	O	O
left	I-10033446	O	O	O	O
leg	I-10033446	O	O	O	O
and	O	O	O	O	O
most	O	O	O	O	O
of	O	O	O	O	O
my	O	O	O	O	O
joints	I-10003239	O	O	O	O
.	O	O	O	O	O